# Load Libraries

In [17]:
import pandas as pd
import json

# Classes

## Segment

In [3]:
class Segment:
    # All parameters are by Month
    def __init__(self,
                 id: str,
                 lon: float, lat: float,
                 areaKm: float,
                 avgTickets: float,
                 avgCustomers: float,
                 avgPackages: float,
                 avgPackagesBySales: float,
                 packages: float,
                 sales: float,
                 customers: int,
                 avgDropSize: float
                 ):
        self.id             =   id
        self.geographyLocation = (lon, lat)
        self.areaKm         =   areaKm
        self.avgTickets     =   avgTickets
        self.avgCustomers   =   avgCustomers
        self.avgPackages    =   avgPackages
        self.avgPackagesBySales =   avgPackagesBySales
        self.packages       = packages
        self.customers      = customers
        self.sales          = sales
        self.avgDropSize    = avgDropSize

## Satelite

In [34]:
class Satelite:
    def __init__(self,
                 id: str,
                 lon: float, lat: float,
                 distanceFromDC: float,
                 durationFromDC: float,
                 durationInTrafficFromDC: float
                 ):
        self.id = id
        self.geographyLocation          = (lon, lat)
        self.distanceFromDC             = distanceFromDC
        self.durationFromDC             = durationFromDC
        self.durationInTrafficFromDC    = durationInTrafficFromDC

# Functions

## Load Distances and Durations Matrix (Segment To Satelite)

In [15]:
def load_distances_duration_matrix(DEBUG = False) -> dict[str,dict]:
    df = pd.read_csv('../others/Levantamiento de Información/Informacion Satelites a Hexagonos.csv')
    size = len(df)
    distance = dict(
        [((df.Satelite[i],df.h3_address[i]),df.loc[i,"distance.value"]/1000) for i in range(size)]
    )
    duration = dict(
        [((df.Satelite[i],df.h3_address[i]),df.loc[i,'duration.value']/3600) for i in range(size)]
    )
    durationInTraffic = dict(
        [((df.Satelite[i],df.h3_address[i]),df.loc[i,'duration_in_traffic.value']/3600) for i in range(size)]
    )
    matrixes = {
        "duration": duration,
        "distance": distance,
        "durationInTraffic": durationInTraffic
    }
    return matrixes

## Load segments

In [25]:
def load_segments(DEBUG = False) -> list[Segment]:
    segments = []
    df = pd.read_csv('../others/Levantamiento de Información/Caracterización de hexagonos de La Paz.csv')
    for i in range(len(df)):
        newSegment = Segment(str(df.h3_address[i]),
                             df.loc[i,'lon.centroid'],
                             df.loc[i,'lat.centroid'],
                             df.loc[i,'area.km2'],
                             df.loc[i,'media.boletas.mes'],
                             df.loc[i,'media.clientes.mes'],
                             df.loc[i,'media.suma.cajas.mes'],
                             df.loc[i,'media.cajas.por.venta'],
                             df.loc[i,'suma.cajas'],
                             df.loc[i,'Cantidad.ventas'],
                             df.loc[i,'Total.clientes'],
                             df.loc[i,'media.cajas.cliente'])
        segments.append(newSegment)
    if DEBUG:
        print("-"*50)
        print("Count of SEGMENTS: ", len(segments))
        print("First Segment:")
        print(json.dumps(segments[0].__dict__,indent=2,default=str))
    return segments

## Load satelites

In [35]:
def load_satelites(DEBUG = False) ->list[Satelite]:
    satelites = []
    df = pd.read_csv('../others/Levantamiento de Información/Informacion CD a Satelites.csv')
    for i in range(len(df)):
        newSatelite = Satelite(str(df.nombre[i]),
                               df.longitud[i],
                               df.latitud[i],
                               df.loc[i,'distance.value']/1000,
                               df.loc[i,'duration.value']/3600,
                               df.loc[i,'duration_in_traffic.value']/3600)
        satelites.append(newSatelite)
    if DEBUG:
        print("-"*50)
        print("Count of SATELITES: ", len(satelites))
        print("First Satelite:")
        print(json.dumps(satelites[0].__dict__,indent=2,default=str))
    return satelites

# Arce

# Models

## Deterministic

In [ ]:
class Model:
    def __init__(self,
                 arce: dict,
                 nameModel ='Deterministic'):
        self.model = ""
